In [1]:
#importing libraries
import os
import numpy as np
import cv2
import os
import shutil
import random
import tensorflow as tf
import imgaug.augmenters as iaa
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model 
from tensorflow.keras.optimizers import Adam


2024-05-20 20:13:50.773866: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 20:13:50.777233: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 20:13:50.822772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 20:13:51.692817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Setting Dataset Path
images_base_directory = "../samaan hamara/Gujarat_Rural_Captchas/"
# images_base_directory = "./my_generated_images"
# images_base_directory = "./final_images"
# Function to get and shuffle image files
def get_and_shuffle_image_files(directory):
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    return image_files



In [3]:
# need to get images names because ismein labels hain

def load_data(image_files):
    images = []
    labels = []
    for file in image_files:
        img = load_img(images_base_directory + "/" +  file, color_mode='grayscale')
        img = img_to_array(img) / 255.0
        label = os.path.basename(file).split('.')[0]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

def get_image_files(directory):
    return os.listdir(directory)


image_files = get_image_files(images_base_directory)    
image_files = get_and_shuffle_image_files(images_base_directory)


In [47]:
print(image_files)

['112665.png', '101757.png', '113640.png', '114638.png', '111283.png', '104600.png', '106075.png', '112861.png', '104608.png', '106502.png', '103733.png', '101895.png', '116147.png', '117537.png', '118186.png', '103093.png', '111675.png', '114437.png', '108372.png', '105534.png', '103529.png', '701998.png', '107606.png', '112039.png', '104325.png', '112497.png', '149865.png', '109075.png', '103030.png', '118969.png', '117535.png', '102349.png', '117521.png', '113297.png', '109630.png', '107831.png', '107197.png', '100898.png', '115147.png', '106160.png', '113833.png', '114498.png', '118094.png', '100437.png', '101565.png', '102673.png', '102324.png', '112588.png', '117510.png', '106801.png', '118307.png', '117276.png', '103627.png', '117206.png', '116219.png', '102849.png', '115512.png', '107671.png', '110299.png', '710649.png', '104275.png', '111533.png', '111931.png', '113274.png', '302212.png', '101312.png', '107738.png', '100482.png', '116547.png', '105120.png', '100466.png', '1001

In [5]:
display(load_data(image_files[0:5])[0].shape)

(5, 80, 190, 1)

### processing labels and creating categorical label

In [6]:
# only loading 80% of data for training
images, labels = load_data(image_files)

# Preprocess the labels

def preprocess_labels(labels, num_characters, num_classes):
    processed_labels = []
    for label in labels:
        # converting individual to int because kaam aayenge
        label = [int(char) for char in label]
        # Convert each integer to categorical
        categorical_label = np.zeros((num_characters, num_classes))
        for i, char in enumerate(label):
            categorical_label[i, char] = 1
        processed_labels.append(categorical_label)
    return np.array(processed_labels)



num_characters = 6  # no of chars
num_classes = 10  # Digits 0-9
labels = preprocess_labels(labels, num_characters, num_classes)


In [7]:
print(labels.shape)

(1774, 6, 10)


In [9]:


def create_model(input_shape, num_classes, num_characters):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_characters * num_classes, activation='softmax'))
    model.add(Reshape((num_characters, num_classes)))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = images[0].shape
model = create_model(input_shape, num_classes, num_characters)

model.fit(images, labels, epochs=32, batch_size=32, validation_split=0.2)
print(input_shape)

Epoch 1/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 816ms/step - accuracy: 0.2894 - loss: 1.8930 - val_accuracy: 0.3019 - val_loss: 1.9995
Epoch 2/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 27s 593ms/step - accuracy: 0.3047 - loss: 1.7815 - val_accuracy: 0.3009 - val_loss: 1.9625
Epoch 3/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 27s 603ms/step - accuracy: 0.3073 - loss: 1.7621 - val_accuracy: 0.3085 - val_loss: 2.0319
Epoch 4/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 730ms/step - accuracy: 0.3143 - loss: 1.7538 - val_accuracy: 0.3099 - val_loss: 1.9437
Epoch 5/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 29s 656ms/step - accuracy: 0.3160 - loss: 1.7405 - val_accuracy: 0.3113 - val_loss: 1.9787
Epoch 6/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 27s 592ms/step - accuracy: 0.3494 - loss: 1.7031 - val_accuracy: 0.3864 - val_loss: 1.7377
Epoch 7/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 726ms/step - accuracy: 0.4290 - loss: 1.5923 - val_accuracy: 0.4671 - val_loss: 1.4550
Epoch 8/32
45/45 ━━━━━━━━━━━━━━━━━━━━ 28s 612ms/step - accuracy: 0.4573 - loss: 1.4474 - val_accu

In [45]:
# resizing and normalizing an image
def preprocess_image(image_file, target_size):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
    if img is None:
        raise ValueError(f"Image {image_file} not found.")
    img = cv2.resize(img, target_size) 
    img = img / 255.0  
    # adding a channel dimension to the last 
    img = np.expand_dims(img, axis=-1) 
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_captcha(model, preprocessed_img):
    prediction = model.predict(preprocessed_img)
    print(prediction[0])
    predicted_label = ''.join([str(np.argmax(char)) for char in prediction[0]])
    return predicted_label

# predicting the New CAPTCHA
image_file = images_base_directory + '/' + "200401.png"
preprocessed_img = preprocess_image(image_file, (190, 80))  # Resize to 190x80
captcha = predict_captcha(model, preprocessed_img)
print(f'Predicted CAPTCHA: {captcha}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[3.5631614e-15 1.8488194e-07 3.0031363e-03 4.8648790e-08 6.7447139e-08
  1.3580507e-08 1.1274724e-09 3.4485035e-09 1.0075590e-09 3.5938719e-10]
 [1.2722381e-02 2.5543359e-09 5.2896066e-08 2.3140467e-08 4.6606382e-13
  1.2442633e-05 6.5815804e-11 3.8824569e-10 7.2568788e-08 9.8959470e-07]
 [3.9891455e-02 2.7210290e-06 2.0447457e-07 7.6344246e-09 2.5732089e-07
  1.4006955e-07 7.9771953e-06 1.9247931e-08 5.5394221e-08 7.3234692e-06]
 [2.2609402e-06 6.4769888e-06 4.7889671e-06 2.4346582e-07 7.0649743e-01
  3.9542002e-07 5.7532992e-07 3.3463673e-06 5.3525732e-11 1.1649904e-06]
 [7.4608013e-02 3.6741736e-07 2.8282082e-06 3.9423168e-07 5.9574165e-07
  9.1943781e-07 2.1973143e-05 8.0961229e-09 3.2967569e-07 5.9575537e-05]
 [9.0395054e-07 1.6307423e-01 7.3583652e-07 1.2866837e-05 4.6020388e-05
  1.2707580e-06 3.6986035e-07 2.1528967e-06 1.2460954e-09 2.5299276e-09]]
Predicted CAPTCHA: 200401


In [46]:
model.save("main_dataset_model.h5")